In [ ]:
import numpy as np

In [5]:
import json

# لیست فایل‌های آپلود شده را نمایش بدهید
from google.colab import files
uploaded = files.upload()


Saving job_opportunities.json to job_opportunities.json
Saving resumes.json to resumes.json


In [6]:
with open('resumes.json', 'r') as f:
    resumes = json.load(f)

with open('job_opportunities.json', 'r') as f:
    jobs = json.load(f)

In [7]:
!pip install spacy
!python -m spacy download en_core_web_lg
import spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
nlp = spacy.load('en_core_web_lg')

# تابع محاسبه embedding از لیست کلمات کلیدی
def get_embedding_from_keywords(keywords):
    doc = nlp(" ".join(keywords))
    return doc.vector

# محاسبه embedding برای رزومه‌ها و شغل‌ها
resume_embeddings = [get_embedding_from_keywords(resume["key_words"]) for resume in resumes]
job_embeddings = [get_embedding_from_keywords(job["key_words"]) for job in jobs]

In [16]:
print(np.array(resume_embeddings).shape)
print(np.array(job_embeddings).shape)

(10, 300)
(10, 300)


In [18]:
# تابع فاصله اقلیدسی
def euclidean_distance(a, b):
    return np.linalg.norm(a - b)

# تطابق n رزومه برتر برای یک شغل
def match_resumes_to_job(job_embedding, resume_embeddings, top_n=3):
    distances = [euclidean_distance(job_embedding, resume_embedding) for resume_embedding in resume_embeddings]
    top_indices = np.argsort(distances)[:top_n]
    return top_indices, [distances[i] for i in top_indices]

In [19]:
# test
job = jobs[0]
job_embedding = job_embeddings[0]

indices, scores = match_resumes_to_job(job_embedding, resume_embeddings)

print(f"\n🔍 Top {len(indices)} Matching Resumes for Job: {job['title']}\n")
for rank, idx in enumerate(indices):
    print(f"#{rank+1} - Resume Title: {resumes[idx]['title']} | Distance: {scores[rank]:.4f}")


🔍 Top 3 Matching Resumes for Job: Neurosurgeon

#1 - Resume Title: Neurosurgeon | Distance: 2.0151
#2 - Resume Title: Journalist (Investigative Reporter) | Distance: 3.2763
#3 - Resume Title: Civil Engineer (Structural) | Distance: 3.2957


In [20]:
import streamlit as st

# Streamlit UI
st.title("🧠 Resume-to-Job Matching App")

selected_job_title = st.selectbox("Select a Job Opening", [job["title"] for job in jobs])

selected_index = next((i for i, job in enumerate(jobs) if job["title"] == selected_job_title), None)
if selected_index is not None:
    selected_job = jobs[selected_index]
    selected_embedding = job_embeddings[selected_index]

    st.markdown(f"### Job Description: {selected_job['text']}")
    st.markdown("---")

    indices, scores = match_resumes_to_job(selected_embedding, resume_embeddings)

    st.markdown("### 🔍 Top Matching Resumes")
    for rank, idx in enumerate(indices):
        st.markdown(f"#### #{rank+1}: {resumes[idx]['title']}")
        st.markdown(f"**Similarity Score (lower is better)**: {scores[rank]:.4f}")
        st.markdown(f"**Summary:** {resumes[idx]['text']}")
        st.markdown("---")

2025-05-16 16:46:07.047 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:46:07.049 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:46:07.053 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:46:07.054 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:46:07.058 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:46:07.060 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:46:07.062 Session state does not function when running a script without `streamlit run`
2025-05-16 16:46:07.064 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 16:46

In [22]:
%%writefile matcher_app.py
!pip install pyngrok
from pyngrok import ngrok

# Kill any previous tunnel
!pkill streamlit

# ایجاد تونل و اجرای اپ
public_url = ngrok.connect(port=8501)
print("Streamlit app is live at:", public_url)

!streamlit run matcher_app.py &

Writing matcher_app.py


In [23]:
!streamlit run matcher_app.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.99.194:8501

  Stopping...


In [26]:
import threading
import time
!pip install pyngrok
from pyngrok import ngrok

# بستن اجراهای قبلی
!pkill streamlit

def run_app():
    !streamlit run matcher_app.py

# راه‌اندازی در ترد جدا
thread = threading.Thread(target=run_app)
thread.start()

# راه‌اندازی ngrok
time.sleep(5)  # صبر برای راه‌اندازی Streamlit
public_url = ngrok.connect(8501)
print(f"✅ Your app is live here: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.99.194:8501



ERROR:pyngrok.process.ngrok:t=2025-05-16T16:54:43+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-16T16:54:43+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-16T16:54:43+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.